In [94]:
# Standard Libraries
import os
import random
import pydicom
from PIL import Image
import yaml
import copy

# Data Manipulation Libraries
import pandas as pd
import numpy as np

# Progress Bar
from tqdm import tqdm

# Machine Learning Libraries
import torch
from torch import nn
from sklearn.model_selection import train_test_split

# YOLO library
from ultralytics import YOLO

# BYOL libraries
from lightly.loss import NegativeCosineSimilarity
from lightly.models.modules import BYOLProjectionHead, BYOLPredictionHead
from lightly.models.utils import deactivate_requires_grad, update_momentum
from lightly.transforms.byol_transform import (
    BYOLTransform,
    BYOLView1Transform,
    BYOLView2Transform,
)
from lightly.utils.scheduler import cosine_schedule

**Define Parameters**

In [95]:
# Disease labels
disease_labels = [
    "Aortic enlargement", "Atelectasis", "Calcification", "Cardiomegaly",
    "Consolidation", "ILD", "Infiltration", "Lung Opacity",
    "Nodule/Mass", "Other lesion", "Pleural effusion", "Pleural thickening",
    "Pneumothorax", "Pulmonary fibrosis"
]

# File path
EXTRACTED_PATH = '/cluster/home/bjorneme/projects/Data/vinbigdata-chest-xray-abnormalities-detection-extracted'

# Define parameters
SEED = 42
NUM_WORKERS =32

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


**Set Seed or Reproducibity**

In [96]:
def seed_everything(seed=SEED):
    """
    Sets the seed to ensure reproducibility.
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Apply the seed
seed_everything()

# **Helper Functions**

In [97]:
def open_dicom_image(path):
    ds = pydicom.dcmread(path)
    img = ds.pixel_array.astype(np.float32)
    img = ((img - img.min()) / (img.max() - img.min()) * 255).astype(np.uint8)
    return Image.fromarray(img)

In [98]:
def convert_annotation_to_yolo(group, img_size_override=None):
    img_path = group.iloc[0]['Path']
    img_width, img_height = open_dicom_image(img_path).size
    
    lines = []
    for _, row in group.iterrows():
        x_min, y_min, x_max, y_max = row[['x_min','y_min','x_max','y_max']]
        x_center = ((x_min + x_max) / 2) / img_width
        y_center = ((y_min + y_max) / 2) / img_height
        box_width = (x_max - x_min) / img_width
        box_height = (y_max - y_min) / img_height
        lines.append(f"{int(row['class_id'])} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}")
    return img_path, lines

In [99]:
def process_split(df, split_name, base_dir):
    for image_id, group in df.groupby('image_id'):
        img_path, yolo_lines = convert_annotation_to_yolo(group)
        img_filename = os.path.basename(img_path)
        ext = os.path.splitext(img_filename)[1].lower()
        img_dir = os.path.join(base_dir, "images", split_name)
        label_dir = os.path.join(base_dir, "labels", split_name)
        os.makedirs(img_dir, exist_ok=True)
        os.makedirs(label_dir, exist_ok=True)
        
        pil_img = open_dicom_image(img_path)
        img_filename = os.path.splitext(img_filename)[0] + ".png"
        pil_img.save(os.path.join(img_dir, img_filename))
        
        label_filename = os.path.splitext(img_filename)[0] + ".txt"
        with open(os.path.join(label_dir, label_filename), "w") as f:
            f.write("\n".join(yolo_lines))

# **Step 1: Load Data**

In [100]:
for split in ['train', 'val']:
    os.makedirs(os.path.join(EXTRACTED_PATH, "images", split), exist_ok=True)
    os.makedirs(os.path.join(EXTRACTED_PATH, "labels", split), exist_ok=True)

print("Processing training set...")
# process_split(train_df, "train", EXTRACTED_PATH)
print("Processing validation set...")
# process_split(val_df, "val", EXTRACTED_PATH)

data_yaml = {
    "train": os.path.join(os.getcwd(), EXTRACTED_PATH, "images", "train"),
    "val": os.path.join(os.getcwd(), EXTRACTED_PATH, "images", "val"),
    "nc": 14,
    "names": disease_labels
}

with open("data.yaml", "w") as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

Processing training set...
Processing validation set...


# **Step 2: Data Preprocessing**

In [101]:
def load_labels(csv_path, image_path):
    """
    Loads and preprocesses the labels from the CSV file.
    Maps each image to its corresponding file path and binary labels for each disease.
    """

    # Read the CSV file containing labels
    labels_df = pd.read_csv(csv_path)

    # Create binary columns for each disease label
    for disease in disease_labels:
        labels_df[disease] = labels_df['class_name'].str.contains(disease).astype(int)
    
    # Create a binary column for 'No Finding'
    labels_df['No finding'] = labels_df['class_name'].apply(lambda x: 1 if 'No finding' in x else 0)

    # Add the full image path
    labels_df['Path'] = labels_df['image_id'].apply(
        lambda x: os.path.join(image_path, 'train', f"{x}.dicom")
    )

    return labels_df

# Path to the labels CSV file
labels_csv_path = os.path.join(EXTRACTED_PATH, 'train.csv')

# Load and preprocess the labels
labels_df = load_labels(labels_csv_path, EXTRACTED_PATH)

**Filter Images with Finding**

In [102]:
labels_df = labels_df[labels_df['class_id'] != 14]

**Split Dataset**

In [103]:
# Split based on image id
unique_ids = labels_df['image_id'].unique()

# Split patients into training and validation
train_ids, val_ids = train_test_split(
    unique_ids, test_size=0.2, random_state=SEED
)

# Create dataframes for training and validation sets
train_df = labels_df[labels_df['image_id'].isin(train_ids)]
val_df = labels_df[labels_df['image_id'].isin(train_ids)]

# Verify Split Sizes
print(f"Train size: {len(train_ids)}")
print(f"Val size: {len(val_ids)}")

Train size: 3515
Val size: 879


# **Step 3: Build the Model**

In [104]:
yolo_model = YOLO("yolo11n.pt")
yolo_model.to(device)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_

In [105]:
# --- Define the BYOL module ---
class BYOL(nn.Module):
    def __init__(self, backbone, in_features):
        super().__init__()
        self.backbone = backbone
        # Use the computed in_features for the projection head.
        self.projection_head = BYOLProjectionHead(in_features, 1024, 256)
        self.prediction_head = BYOLPredictionHead(256, 1024, 256)

        self.backbone_momentum = copy.deepcopy(self.backbone)
        self.projection_head_momentum = copy.deepcopy(self.projection_head)

        deactivate_requires_grad(self.backbone_momentum)
        deactivate_requires_grad(self.projection_head_momentum)

    def forward(self, x):
        y = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(y)
        p = self.prediction_head(z)
        return p

    def forward_momentum(self, x):
        y = self.backbone_momentum(x).flatten(start_dim=1)
        z = self.projection_head_momentum(y)
        return z.detach()

class PoolHead(nn.Module):
  """ Apply average pooling to the outputs. Adapted from Classify head."""
  def __init__(self, f, i, c1):
    super().__init__()
    self.f = f  # receive the outputs from these layers
    self.i = i  # layer number
    self.conv = Conv(c1, 1280, 1, 1, None, 1)
    self.avgpool = nn.AdaptiveAvgPool2d(1)

  def forward(self, x):
    return self.avgpool(self.conv(x))

# Backbone
yolo_model.model.model = yolo_model.model.model[:12]

# Determine the backbone output dimension
dummy_input = torch.randn(1, 3, 224, 224).to(device)

# Run forward pass only using backbone
out = yolo_model.model(dummy_input)
input_dim = out.flatten(start_dim=1).shape[1]

print("Backbone output dimension:", input_dim)

backbone = yolo_model.model.requires_grad_()
backbone.train()

# Create BYOL model with the corrected in_features:
byol_model = BYOL(backbone, input_dim)
byol_model.to(device)

Backbone output dimension: 50176


BYOL(
  (backbone): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_runni

In [106]:
byol_transform = BYOLTransform()

In [107]:
import torchvision

path = os.path.join(EXTRACTED_PATH, "images")

dataset = torchvision.datasets.ImageFolder(path, transform=byol_transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True, num_workers=32)

# --- BYOL pretraining loop ---
criterion = NegativeCosineSimilarity()
optimizer = torch.optim.SGD(byol_model.parameters(), lr=0.5)
epochs = 1
print("Starting BYOL pretraining")
for epoch in range(epochs):
    total_loss = 0

    progress_bar = tqdm(dataloader, desc=f"BYOL Epoch {epoch+1}/{10}")

    momentum_val = cosine_schedule(epoch, epochs, 0.996, 1)
    for (x0, x1), _ in progress_bar:  # ignore labels from ImageFolder
        # Update momentum versions of the backbone and projection head
        update_momentum(byol_model.backbone, byol_model.backbone_momentum, m=momentum_val)
        update_momentum(byol_model.projection_head, byol_model.projection_head_momentum, m=momentum_val)
        x0, x1 = x0.to(device), x1.to(device)
        p0, p1 = byol_model(x0), byol_model(x1)
        z0, z1 = byol_model.forward_momentum(x0), byol_model.forward_momentum(x1)
        loss = 0.5 * (criterion(p0, z1) + criterion(p1, z0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        progress_bar.set_postfix({"Loss": loss.item()})
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch:02d} Loss: {avg_loss:.5f}")

Starting BYOL pretraining


BYOL Epoch 1/10:   0%|          | 0/137 [00:00<?, ?it/s]

BYOL Epoch 1/10: 100%|██████████| 137/137 [00:21<00:00,  6.47it/s, Loss=-0.582]

Epoch 00 Loss: -0.57189


In [108]:
# Transfer weights from pretrained model
yolo_model.model.load(model.student_backbone)

NameError: name 'model' is not defined

# **Step 4: Train the Model**

In [ ]:
# Train with image size 224 (instead of 640)
results = yolo_model.train(data="data.yaml", epochs=100, imgsz=224)

New https://pypi.org/project/ultralytics/8.3.92 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show

E0000 00:00:1742316643.909663 2421531 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742316643.915370 2421531 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Overriding model.yaml nc=80 with nc=14

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytic

train: Scanning /cluster/home/bjorneme/projects/Data/vinbigdata-chest-xray-abnormalities-detection-extracted/labels/train.cache... 3515 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3515/3515 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

                   all        879       7139      0.424      0.189      0.169     0.0825

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     12/100      1.96G      1.692       1.66      1.207        152        224: 100%|██████████| 220/220 [00:15<00:00, 14.34it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:03<00:00,  8.75it/s]


                   all        879       7139      0.355      0.192       0.16     0.0772

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     13/100      1.96G      1.675      1.656      1.207        231        224:  64%|██████▎   | 140/220 [00:09<00:05, 14.25it/s]


KeyboardInterrupt: 